## Load data from .odt format

In [16]:
from odf.opendocument import load
from odf import text, teletype

raw_data = []
textdoc = load('dataset/data.odt')
allparas = textdoc.getElementsByType(text.P)
for line in allparas:
    raw_data.append(teletype.extractText(line))

In [18]:
while '' in raw_data:
    raw_data.remove('')
raw_data

['Wypowiedzi OG',
 'Przywitanie',
 '- Dzień dobry.',
 '[',
 'dzień dobry',
 'witać',
 ']',
 'Wyjaśnienie powodu wizyty',
 '- Chcę złożyć wniosek o wydanie dowodu osobistego.',
 '[',
 'ja wniosek dowód1 mieć',
 'ja wniosek dowód2 mieć',
 ']',
 '- Czy mogę odebrać dowód?',
 '[',
 'czy już nowy dowód1',
 'czy już nowy dowód2',
 'być mój nowy dowód1',
 'być mój nowy dowód2',
 ']',
 '- Chcę zgłosić utratę dowodu osobistego.',
 '[',
 'ja dowód1 zgubić',
 'ja dowód2 zgubić',
 ']',
 '- Chcę złożyć wniosek o wydanie dowodu osobistego dla mojego dziecka.',
 '[',
 'ja dowód1 moje dziecko chcieć',
 'ja dowód2 moje dziecko chcieć',
 ']',
 '- Chcę złożyć wniosek o wydanie dowodu osobistego dla mojej żony.',
 '[',
 'ja dowód1 moja żona chcieć',
 'ja dowód2 moja żona chcieć',
 ']',
 'Obsługa',
 '- Zmieniłem adres.',
 '[',
 'dom przenieść się',
 'mieszkanie zmiana1',
 'mieszkanie zmiana2',
 'adres1 zmiana1',
 'adres2 zmiana1',
 'adres1 zmiana2',
 'adres2 zmiana2',
 ']',
 '- Upłynął termin ważności dowo